<a href="https://colab.research.google.com/github/namvcpp/ACM_Notebook_new/blob/master/translate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch

In [2]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [3]:
tokenizer_en2vi = AutoTokenizer.from_pretrained("vinai/vinai-translate-en2vi-v2", src_lang="en_XX")
model_en2vi = AutoModelForSeq2SeqLM.from_pretrained("vinai/vinai-translate-en2vi-v2")

device_en2vi = torch.device(torch.device("cuda" if torch.cuda.is_available() else "cpu"))
model_en2vi.to(device_en2vi)

config.json:   0%|          | 0.00/1.41k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/1.41M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/1.69G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

MBartForConditionalGeneration(
  (model): MBartModel(
    (shared): MBartScaledWordEmbedding(66773, 1024, padding_idx=1)
    (encoder): MBartEncoder(
      (embed_tokens): MBartScaledWordEmbedding(66773, 1024, padding_idx=1)
      (embed_positions): MBartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x MBartEncoderLayer(
          (self_attn): MBartAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)


In [7]:
# CPU-based
def translate_en2vi(en_text: str) -> str:
    input_ids = tokenizer_en2vi(en_text, return_tensors="pt").input_ids
    output_ids = model_en2vi.generate(
        input_ids,
        decoder_start_token_id=tokenizer_en2vi.lang_code_to_id["vi_VN"],
        num_return_sequences=1,
        num_beams=5,
        early_stopping=True
    )
    vi_text = tokenizer_en2vi.batch_decode(output_ids, skip_special_tokens=True)
    vi_text = " ".join(vi_text)
    return vi_text

In [4]:
# GPU-based
def translate_en2vi(en_texts: str) -> str:
    input_ids = tokenizer_en2vi(en_texts, padding=True, return_tensors="pt").to(device_en2vi)
    output_ids = model_en2vi.generate(
        **input_ids,
        decoder_start_token_id=tokenizer_en2vi.lang_code_to_id["vi_VN"],
        num_return_sequences=1,
        num_beams=5,
        early_stopping=True
    )
    vi_texts = tokenizer_en2vi.batch_decode(output_ids, skip_special_tokens=True)
    return vi_texts

In [5]:
# The input may consist of multiple text sequences, with the number of text sequences in the input ranging from 1 up to 8, 16, 32, or even higher, depending on the GPU memory.
en_texts = ['''Cid was part of the first wave but ventured in a different direction to find the leader.''',
'''He grumbles about being lost in the labyrinthine base.''',
'''Sensing an approaching person, he poses himself to welcome the straggler.''',
'''Introductions are cut short as the muscular frame lunges at Cid, but a quick lateral parry prevents the lethal blow.''',
'''Every exchange only increases the shadowbroker’s contempt toward the man.''',
'''The anonymous dark knight’s unsophisticated movements depend on brute force, neither understanding nor appreciating the intricacies of battle.''',
'''Cid decides to instruct the mass of muscles in proper combat.''']

en_text = '''Cid was part of the first wave but ventured in a different direction to find the leader. He grumbles about being lost in the labyrinthine base. Sensing an approaching person, he poses himself to welcome the straggler. Introductions are cut short as the muscular frame lunges at Cid, but a quick lateral parry prevents the lethal blow. Every exchange only increases the shadowbroker’s contempt toward the man. The anonymous dark knight’s unsophisticated movements depend on brute force, neither understanding nor appreciating the intricacies of battle. Cid decides to instruct the mass of muscles in proper combat.'''

In [6]:
print(translate_en2vi(en_texts))

['Cid là một phần của làn sóng đầu tiên nhưng mạo hiểm theo một hướng khác để tìm ra thủ lĩnh.', 'Ông càu nhàu về việc bị lạc trong căn cứ mê cung.', 'Cảm nhận được một người đang đến gần, anh ta tự tạo dáng để chào đón người đi lạc.', 'Giới thiệu được cắt ngắn khi khung cơ bắp lao vào Cid, nhưng một parry bên nhanh chóng ngăn chặn đòn gây chết người.', 'Mỗi cuộc trao đổi chỉ làm tăng sự khinh miệt của kẻ môi giới bóng tối đối với người đàn ông.', 'Phong trào tinh vi của Kỵ sĩ bóng đêm vô danh phụ thuộc vào sức mạnh vũ phu, không hiểu cũng không đánh giá cao sự phức tạp của trận chiến.', 'Cid quyết định hướng dẫn khối lượng cơ bắp trong chiến đấu thích hợp.']
